### Business Statement:
To Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.
We will be provided a company name and their primary website.

In [1]:
# Importing Libraries

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI